In [1]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets

In [2]:
!pip install optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.0/410.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
df2 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_projections.csv')
df1 = pd.read_csv('/kaggle/input/chest-xrays-indiana-university/indiana_reports.csv')
images_captions_df = pd.DataFrame({'imgs': [],
                                    'captions': []})
for i in range(len(df2)):
    uid = df2.iloc[i]['uid']
    image = df2.iloc[i]['filename']
    index = df1.loc[df1['uid'] ==uid]
    
    if not index.empty:    
        index = index.index[0]
        caption = df1.iloc[index]['findings']
        if type(caption) == float:
         
            continue 
        images_captions_df = pd.concat([images_captions_df, pd.DataFrame([{'imgs': image, 'captions': caption}])], ignore_index=True)
p = '/kaggle/input/chest-xrays-indiana-university/images/images_normalized/'
images_captions_df['imgs'] = p+ images_captions_df['imgs']
images_captions_df.head()

,imgs,captions
0,/kaggle/input/chest-xrays-indiana-university/i...,The cardiac silhouette and mediastinum size ar...
1,/kaggle/input/chest-xrays-indiana-university/i...,The cardiac silhouette and mediastinum size ar...
2,/kaggle/input/chest-xrays-indiana-university/i...,Borderline cardiomegaly. Midline sternotomy XX...
3,/kaggle/input/chest-xrays-indiana-university/i...,Borderline cardiomegaly. Midline sternotomy XX...
4,/kaggle/input/chest-xrays-indiana-university/i...,There are diffuse bilateral interstitial and a...


In [4]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class ImageCaptioningDataset(Dataset):
    def __init__(self, df,processor):
        self.images = df['imgs'].values
        self.captions = df['captions'].values
        self.processor = processor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = str(self.images[idx])
        image = Image.open(image_path).convert("RGB")
        caption = self.captions[idx]
       
        encoding = self.processor(images=image, padding="max_length", return_tensors="pt")
        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = caption
        return encoding

def collate_fn(batch):
    # pad the input_ids and attention_mask
    processed_batch = {}
    for key in batch[0].keys():
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor.tokenizer(
                [example["text"] for example in batch], padding=True, return_tensors="pt"
            )
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch


In [41]:
from transformers import Blip2ForConditionalGeneration, AutoProcessor
from peft import PeftModel, PeftConfig

peft_model_id = "shorok31/BLIP_LORA_2"
config = PeftConfig.from_pretrained(peft_model_id)

model = Blip2ForConditionalGeneration.from_pretrained(config.base_model_name_or_path, load_in_8bit=True, device_map="auto")
model = PeftModel.from_pretrained(model, peft_model_id)

processor = AutoProcessor.from_pretrained("shorok31/BLIP_LORA_2")

adapter_config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

In [30]:
# from transformers import AutoProcessor, Blip2ForConditionalGeneration

# processor = AutoProcessor.from_pretrained("shorok31/BLIP_LORA")
# model = Blip2ForConditionalGeneration.from_pretrained("shorok31/BLIP_LORA", device_map="auto", load_in_8bit=True)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
# !pip install accelerate

In [42]:
from peft import LoraConfig, get_peft_model

# Let's define the LoraConfig
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "k_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 5,242,880 || all params: 3,749,922,816 || trainable%: 0.13981301102065136


In [43]:
images_captions_df = images_captions_df.sample(frac=0.5, random_state=42) 

from sklearn.model_selection import train_test_split

train_df,val_df =train_test_split(images_captions_df , test_size=0.2, shuffle=True, random_state=42)

train_dataset = ImageCaptioningDataset(train_df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)



In [44]:
val_df.head()

,imgs,captions
2318,/kaggle/input/chest-xrays-indiana-university/i...,Cardiac silhouette and pulmonary vascularity a...
1789,/kaggle/input/chest-xrays-indiana-university/i...,There is a calcified granuloma in the left upp...
3018,/kaggle/input/chest-xrays-indiana-university/i...,The cardiac and mediastinal silhouettes are un...
1187,/kaggle/input/chest-xrays-indiana-university/i...,"The heart, pulmonary XXXX and mediastinum are ..."
4231,/kaggle/input/chest-xrays-indiana-university/i...,Heart size and mediastinal contour are normal....


In [45]:
val_dataset = ImageCaptioningDataset(val_df, processor)
val_dataloader = DataLoader(val_dataset, shuffle=True, batch_size=4, collate_fn=collate_fn)

In [46]:
import torch
import numpy as np
import tqdm

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

device = "cuda" if torch.cuda.is_available() else "cpu"

model.train()

for epoch in range(1,11):
    epoch_loss = []
    
    for idx, batch in tqdm.tqdm(enumerate(train_dataloader)):
        
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device, torch.float16)

        outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)
    
        loss = outputs.loss

        epoch_loss.append( loss.item())

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        if( idx %100 ==0):
            print(f"Batch: {idx} , Loss: {np.mean(epoch_loss)}")
    print(f"Epoch: {epoch}, Loss:{np.mean(epoch_loss) }")

1it [00:01,  1.97s/it]

Batch: 0 , Loss: 5.12890625


101it [02:19,  1.36s/it]

Batch: 100 , Loss: 2.2465965346534653


201it [04:36,  1.38s/it]

...

Batch: 0 , Loss: 0.96142578125


101it [02:16,  1.35s/it]

Batch: 100 , Loss: 1.0707040918935644


166it [03:44,  1.35s/it]


KeyboardInterrupt: 

In [47]:
# prepare image for the model

device = "cuda" if torch.cuda.is_available() else "cpu"
def generate_caption(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)
    pixel_values = inputs.pixel_values

    generated_ids = model.generate(pixel_values=pixel_values, max_length=1024)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_caption

print(generate_caption(images_captions_df['imgs'][0]))

KeyError: 0

In [ ]:
import tqdm 


predicted_captions = [] 
for i in tqdm.tqdm( val_df['imgs'].values[0:6]):
    predicted_captions.append(generate_caption(i))
print(len(predicted_captions))

  0%|          | 0/6 [00:00<?, ?it/s]The `language_model` is not in the `hf_device_map` dictionary and you are running your script in a multi-GPU environment. this may lead to unexpected behavior when using `accelerate`. Please pass a `device_map` that contains `language_model` to remove this warning. Please refer to https://github.com/huggingface/blog/blob/main/accelerate-large-models.md for more details on creating a `device_map` for large models.


In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Assuming you have a list of predicted captions and a list of ground truth captions
generated_captions = predicted_captions
ground_truth_captions = val_df['captions'].values[0:6]
# Convert the caption lists into the format expected by nltk
ground_truth_captions = [[caption.split() for caption in captions] for captions in ground_truth_captions]
generated_captions = [caption.split() for caption in generated_captions]


# Define the smoothing function to use
# smoothie = SmoothingFunction().method4

# Compute the BLEU score with smoothing
weights = (0.25, 0.25, 0.25, 0.25)  # equal weights for 1-4 gram BLEU scores
score = corpus_bleu(ground_truth_captions, predicted_captions,weights =weights)
print(f'The BELU Score Is: {score}')

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [38]:
from huggingface_hub import notebook_login

notebook_login()

In [39]:
model.push_to_hub("shorok31/BLIP_LORA_2")
processor.push_to_hub("shorok31/BLIP_LORA_2")

adapter_model.safetensors:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shorok31/BLIP_LORA_2/commit/d3210f2628c56e64b77585090c4b5adba98a3532', commit_message='Upload processor', commit_description='', oid='d3210f2628c56e64b77585090c4b5adba98a3532', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model.save()